## Graphs Isomorphism Analysis

In [ ]:
from graph_iso.graph import Graph
from graph_iso.isomorphism import IsomorphismChecker
from graph_iso.parser import GraphParser
from graph_iso.visualization import GraphVisualizer

In [22]:
graphs = GraphParser("../data/sample_graphs.json").load_multiple_graphs()
graph1 = graphs["graph1"]
graph2 = graphs["graph2"]

In [23]:
checker = IsomorphismChecker(graph1=graph1, graph2=graph2)
result, trace_states = checker.trace_states()
print(f"Are graphs isomorphic? {result}")
print(f"Recorded {len(trace_states)} tracing steps.")

Are graphs isomorphic? False
Recorded 333 tracing steps.


In [24]:
visualizer = GraphVisualizer(graph1, graph2, trace_states)
visualizer.run()